In [95]:
import os
import json
import numpy as np


In [96]:
results_dir = "../results"
exp = "discovery"
goal = "direct_discovery"
model = "gpt-4o-boxloop"
seeds = [1,2,3,4,5] 

for env in [
    "hyperbolic_temporal_discount",
    "location_finding",
    "death_process",
    "irt",
    "dugongs",
    "survival",
    "peregrines"]:
    
    

    try:
        files = [os.path.join(results_dir, env, f"{goal}_{model}_{exp}_true_{seed}.json") for seed in seeds]
        files_no_prior = [os.path.join(results_dir, env, f"{goal}_{model}_{exp}_false_{seed}.json") for seed in seeds]
        regret_files = [os.path.join(results_dir, env, f"regret_{goal}_{model}_{exp}_true_{seed}.json") for seed in seeds]
        regret_files_no_prior = [os.path.join(results_dir, env, f"regret_{goal}_{model}_{exp}_false_{seed}.json") for seed in seeds]

        data = []
        for file in files:
#             print(file)
            with open(file, "r") as f:
                data.append(json.load(f))

        data_no_prior = []
        for file in files_no_prior:
#             print(file)
            with open(file, "r") as f:
                data_no_prior.append(json.load(f))

   
        errs, stds = [], []
        for d in data:
            errs.append([])
            stds.append([])
            for r in d['data']['results']: 
                errs[-1].append(r[0][0])
                stds[-1].append(r[0][1])
                
        errs_no_prior, stds_no_prior = [], []
        for d in data_no_prior:
            errs_no_prior.append([])
            stds_no_prior.append([])
            for r in d['data']['results']: 
                errs_no_prior[-1].append(r[0][0])
                stds_no_prior[-1].append(r[0][1])

        errs = np.array(errs)
        stds = np.array(stds)
        
        errs_no_prior = np.array(errs_no_prior)
        stds_no_prior = np.array(stds_no_prior)
        
        if env == "hyperbolic_temporal_discount" and "direct" in goal:
            errs = 1 - errs
            errs_no_prior = 1 - errs_no_prior
        elif env == "peregrines":
            errs = errs ** 0.5
            errs_no_prior = errs_no_prior ** 0.5
        elif env == "survival":
            errs = 1 - errs
            errs_no_prior = 1 - errs_no_prior

        mean_score = np.mean(errs, axis=0)
        ci_95 = 1.96 * np.std(errs, axis=0) / np.sqrt(len(errs))
        mean_score_no_prior = np.mean(errs_no_prior, axis=0)
        ci_95_no_prior = 1.96 * np.std(errs_no_prior, axis=0) / np.sqrt(len(errs_no_prior))

        print(f"env: {env}, {len(errs)} \n")
        print(f"prior mean_score: {mean_score}: {ci_95} \n")
        print(f"no prior mean_score: {mean_score_no_prior}: {ci_95_no_prior} \n")  
        
        
    except Exception as e:
        print(e)
        continue

env: hyperbolic_temporal_discount, 5 

prior mean_score: [0.34]: [0.14242067] 

no prior mean_score: [0.38]: [0.19521443] 

env: location_finding, 5 

prior mean_score: [57.51216762]: [42.72005076] 

no prior mean_score: [12.25686946]: [11.28323861] 

env: death_process, 5 

prior mean_score: [332.14]: [161.75683809] 

no prior mean_score: [317.86]: [238.59025562] 

env: irt, 5 

prior mean_score: [0.46]: [0.19678244] 

no prior mean_score: [0.6]: [0.19988156] 

env: dugongs, 5 

prior mean_score: [0.09052431]: [0.04436874] 

no prior mean_score: [0.12501436]: [0.03291396] 

env: survival, 5 

prior mean_score: [0.7]: [0.12396128] 

no prior mean_score: [0.42]: [0.10222109] 

env: peregrines, 5 

prior mean_score: [132.42163594]: [81.77654771] 

no prior mean_score: [60.56956442]: [57.20533152] 

